In [13]:
from Levenshtein import distance as levenshtein_distance
from time import *
import time
import numpy as np
import math
import random
import os
import pandas as pd

In [14]:
from ipynb.fs.full._1EA_algorithm_v2v import *

# Division Kmer

In [15]:
def kmers_split(seq, k, taille_V, nb_indice):
    dict1 = []
    # on divise nos k-mers en sous partie. le but est de comparer tous les k-mers avec le meme indice=div_counter
    # dans la sequence query avec tous les k-mers avec le meme indice dans la seq ref
    # cela permettra d eviter a comparer les k-mers en premier avec des k-mers en dernier
    
    #nb_indice: exemple -> div = int(280/70) = 4, alors pr chaque 4 kmers on fait +1 a l indice 
    div = int(taille_V/nb_indice)
    div_fix = int(taille_V/nb_indice)
    div_counter = 0
    for i in range(len(seq)-(k-1)):
        if i == div:
            div_counter += 1
            div += div_fix
        dict1.append([seq[i:i+k], div_counter]) 
    #print("div", div)
    return dict1

In [16]:
def kmers_split(seq, k, nb_indice):
    """
    Split kmers
    Input1 seq      : sequence to split
    Input2 k        : length of kmers
    Input3 nb_indice: Number of kmer with same index
    Output list_    : list containing a list of kmer with their indexes 
    """
    list_ = []
    
    # on divise nos k-mers en sous partie. le but est de comparer tous les k-mers avec le meme indice=div_counter
    # dans la sequence query avec tous les k-mers avec le meme indice dans la seq ref
    # cela permettra d eviter a comparer les k-mers en premier avec des k-mers en dernier
    
    div = nb_indice
    div_counter = 0
    
    for i in range(len(seq)-(k-1)): 
        if i == div:
            div_counter += 1
            div += nb_indice
        list_.append([seq[i:i+k], div_counter])   
    return list_

# Etude sur nos Fichiers

In [17]:
# extraction des genes ref
db_path = "human_germline//IGHJ.fasta"
dic_ref_j = dict(read_db(db_path))

#print(dic_ref_j['IGHJ1*01'])

# Comparaison des listes de kmer de la forme [[kmer,indice]]

In [18]:
import time

def comparaisonJ(query_kmer, ref_kmer, l_taille, bloc_max):
    count = 0 # pour le nombre de [kmer, indice] identique dans les deux sequences
    block = 0 # pour areter l'algorithme si pour un nombre 'block' on n'a pas trouver des [kmer, indice] identique
    query_kmer_copy = query_kmer.copy()
    last_pos_query = 0 # dernier element de query qui a ete compare

    for kmer in ref_kmer: # on parcout les [kmer, indice] de la sequence reference
        if kmer in query_kmer_copy: # on voit si ces [kemr, indice] sont dans la sequence query
            count += 1 # on precise qu'on a trouve un [kmer, indice]
            block = 0 # on remet a 0 le block vu qu'on a trouver un [kemr, indice]
            query_kmer_copy.remove(kmer)
            last_pos_query = query_kmer.index(kmer)
                     
        if block == bloc_max:
            position = l_taille - (5 + len(query_kmer[0:last_pos_query])) # k = 5
            return count, position
        #print(kmer)
        
        block += 1
    index_dernier_element = query_kmer.index(query_kmer[-1])
    
    position = l_taille - (5 + len(query_kmer[0:index_dernier_element])) # k = 5
    return count, position
#debut = time.time()
#comparaisonJ(q_kmer, r_kmer, l_taille, 40)
#comparaisonJ(q_kmer, r_kmer, l_taille, 40)
#comparaisonJ(q_kmer, r_kmer, l_taille, 40)
#fin = time.time()
#print("temps:", fin-debut)

In [19]:
# fonction qui nous permet de comparer les [kemrs, indice] des seq ref avec les [kmer, indice] 
# d'une seule seq query
def compare_fichierJ(dic_db_keys, dic_db_values, q_kmer, bloc_max, nb_indice, l_taille, k):
    dic = {}
    dic_position = {}
    for j in range(len(dic_db_keys)):
        r_kmer = kmers_split(dic_db_values[j][::-1], k, nb_indice) # on divise en [kmer, indice] la seq ref
        comp, position = comparaisonJ(r_kmer, q_kmer, l_taille[j], bloc_max ) # on realise la comparaison
        cle = dic_db_keys[j]
        dic[cle] = comp
        dic_position[cle] = position
            
    return dic, dic_position

In [20]:
query_IGHJ1_01 = 'GACACGGCTGTGTATTACTGTGCGAGCGTTAAATGTGGATATAGTGGCTACGATTACACATACTTCCAGCACTGGGGCCAGGGCACCCTGGTCACCGTCTCCTCAG'

ref_IGHJ1_01 = 'gctgaatacttccagcactggggccagggcaccctggtcaccgtctcctcag'
ref_IGHJ2_02 = 'ctactggtacttcgatctctggggccgtggcaccctggtcactgtctcctcag'

k = 5
taille_v = 280
nb_indice = 70

q_kmerj = kmers_split(query_IGHJ1_01[::-1], k, nb_indice)
r_kmerj = kmers_split(ref_IGHJ1_01[::-1].upper(), k, nb_indice)
r_kmerj2 = kmers_split(ref_IGHJ2_02[::-1].upper(), k, nb_indice)
bloc_max = 40
#print(comparaison(q_kmerj, r_kmerj, bloc_max))
#print(comparaison(q_kmerj, r_kmerj2, bloc_max))

## Partie J

In [21]:
query_IGHJ1_01 = 'GACACGGCTGTGTATTACTGTGCGAGCGTTAAATGTGGATATAGTGGCTACGATTACACATACTTCCAGCACTGGGGCCAGGGCACCCTGGTCACCGTCTCCTCAG'

ref_IGHJ1_01 = 'gctgaatacttccagcactggggccagggcaccctggtcaccgtctcctcag'
ref_IGHJ2_02 = 'ctactggtacttcgatctctggggccgtggcaccctggtcactgtctcctcag'

k = 5
taille_j = 280
nb_indice = 70

q_kmerj = kmers_split(query_IGHJ1_01[::-1], k, nb_indice)
r_kmerj = kmers_split(ref_IGHJ1_01[::-1].upper(), k, nb_indice)
r_kmerj2 = kmers_split(ref_IGHJ2_02[::-1].upper(), k, nb_indice)

bloc_max = 40
#print(comparaisonJ(q_kmerj, r_kmerj, l_taille, 40))
#print(comparaisonJ(q_kmerj, r_kmerj2, l_taille, 40))

In [22]:
import time
def main_codeJ(dic_db, jquery_keys, jquery_values, nb_indice = 60, bloc_max = 20, i =0, taille_V = 20, while_i = 2):

    #debut_algo = time.time()
    # gene ref
    dic_db_keys = list(dic_db.keys()) # on prend le nom du gene ref
    dic_db_values = list(dic_db.values()) # one prend la sequence du gene ref
    
    
    dic = {}
    tot_seq = 0 # le nombre de seqeunces etudiees
    tot_vrai = 0 # le nombre de vrais positifs
    temps_max = 0
    temps_levenshtein = 0
    
    # query
    #print(len(jquery_keys))
    #print(len(jquery_values))
    dic_pos = {}
    l_pos_max = []
    
    while i < while_i:
        
        tmpj = jquery_keys[i]
        #print(">"+tmpj)
        queryj = jquery_values[i]
        #print(queryj.lower())
        q_kmer = kmers_split(queryj[::-1], 5, nb_indice) # on divise en [kmer, indice] la seq query

        debut_comp = time.time()
        # on commence l'algorithme
        dic, dic_pos = compare_fichierJ(dic_db_keys, dic_db_values, q_kmer, bloc_max, nb_indice, l_taille, 5)
        fin_comp = time.time()
        temps_comp = fin_comp - debut_comp
        #print(dic_position)
        max_ = max(dic.items(), key=lambda x: x[1]) # on prend le score max des comparaisons faites
        listOfKeys = list()
        listOfRefs = list()
        
        # on teste s'il y a plusieurs gene ref qui ont le meme score et on les mets dans une liste
        for key, value in dic.items():
            if value == max_[1]:
                listOfKeys.append(key)
                listOfRefs.append(dic_db[key])
        
        distance = 0
        distance_max = 0
        max_dic = ''
        
        # s'il y a bien plusieurs gene ref dans la liste alors on utilse la fct levenshtein_distance pour
        # essayer de trouver la bonne sequence
        dict_count = {} # pour compter le nombre de fois où on a le même gene d allèles differents
        dict_count, cle, cle2 = ref_in_listoOfKeys(listOfKeys, listOfRefs, queryj)
               
        if len(listOfKeys) == 1:
            temps_levenshtein = 0
            if cle[-1] == 'D': # on elenve les duplicer
                max_dic = cle[:-1]
            max_dic = cle
            max_dic_avec_cle = cle2
            
        
        if len(listOfKeys) > 1 :
            debut_levenshtein = time.time()
            dict_count, max_dic, cle, max_dic_avec_cle = ref_in_listoOfKeys_distance(listOfKeys, listOfRefs, queryj, distance_max)
                    
            fin_levenshtein = time.time()
            temps_levenshtein = fin_levenshtein - debut_levenshtein
        
        l_pos_max.append(dic_pos[max_dic_avec_cle])
        # on enleve le '*' de la seq ref et tout ce qui est apres le '*
        tmpj = remove_etoile(tmpj)
        if tmpj[-1] == 'D': # on elenve les duplicer
            tmpj = tmpj[:-1]
        
        """
        print("Sequences avec même score :", listOfKeys)
        print("Nombre d'iteration        :", i)
        print("Sequence reference        :", max_dic)
        print("Sequence query            :", tmpj)
        print("Temps levenshtein         :", temps_levenshtein)
        print("Temps complet:            :", temps_comp)
        """
        temps_max += temps_comp
            
        
        # on test si on a des vrais positifs
        if max_dic == tmpj: # si c'est des  vrais positifs
            #print("-> VRAI Même sequence trouvée")
            tot_vrai += 1
        else:
            lllll = 0
            #print("-> FAUX Différente sequence trouvée")
        #print("-----------------------------------------------------")
        
        dic = {}
        tot_seq += 1
        i += 1
    vrai_pos = (tot_vrai/tot_seq)*100
    
    m, s = divmod(int(temps_max), 60)
    h, m = divmod(m, 60)
    
    print("                                 Résultat final                                  ")
    print("---------------------------------------------------------------------------------")
    print("Nom du fichier ref                         :", db_path)
    print("Nom du fichier query                       :", name_fichier_jquery[46:len(name_fichier_jquery)])
    print("Nombre de vrais positifs                   :", tot_vrai)
    print("Nombre de séquences testées                :", tot_seq)
    print("Pourcentage de vrais positifs pour les V   :", vrai_pos, "%")
    #print("Temps final de l'algorithme                :", strftime('%H %M %S', gmtime(temps_max)))
    #print("Temps final de l'algorithme                :", f'{h:d}:{m:02d}:{s:02d}')
    #print("-----------------------------------------------------")
    #print("-----------------------------------------------------")
    #print(l_pos_max)
    return l_pos_max

In [23]:
# nb_indice = 500, bloc_max = 30, pourcentage = 69.96% 
#print(l_finalJ)

In [24]:
#l_finalJ = [328, 322, 319, 333, 308, 322, 350, 315, 304, 322, 321, 316, 323, 305, 305, 307, 317, 319, 308, 311, 316, 308, 335, 328, 316, 313, 346, 345, 335, 314, 310, 311, 320, 316, 324, 334, 326, 329, 322, 331, 321, 318, 332, 311, 312, 341, 321, 351, 316, 290, 349, 308, 335, 327, 311, 319, 314, 310, 310, 327, 325, 320, 294, 315, 323, 331, 291, 304, 322, 307, 310, 313, 321, 311, 327, 308, 285, 284, 340, 321, 337, 326, 348, 311, 322, 333, 334, 311, 360, 335, 309, 321, 304, 331, 316, 343, 324, 310, 304, 323]